This notebook contains a minimal example of using `fev` to evaluate time series forecasting models.

In [1]:
import fev

/home/shchuro/envs/fev/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Create a task from a dataset stored on Hugging Face Hub
task = fev.Task(
    dataset_path="autogluon/chronos_datasets",
    dataset_config="ercot",
    horizon=24,
    num_windows=2,
)

In [3]:
# A task consists of multiple rolling evaluation windows
for window in task.iter_windows():
    print(window)

Validating dataset format (num_proc=8): 100%|██████████| 8/8 [00:00<00:00, 71.32 examples/s]

EvaluationWindow(cutoff=-48, horizon=24, min_context_length=1, max_context_length=None, id_column='id', timestamp_column='timestamp', target_columns=['target'], known_dynamic_columns=[], past_dynamic_columns=[], static_columns=[])
EvaluationWindow(cutoff=-24, horizon=24, min_context_length=1, max_context_length=None, id_column='id', timestamp_column='timestamp', target_columns=['target'], known_dynamic_columns=[], past_dynamic_columns=[], static_columns=[])


In [4]:
# Load data available as input to the forecasting model
past_data, future_data = task.get_window(0).get_input_data()

In [5]:
# past data before the forecast horizon.
past_data

Dataset({
    features: ['id', 'timestamp', 'target'],
    num_rows: 8
})

In [6]:
past_data[0]

{'id': np.str_('COAST'),
 'timestamp': array(['2004-01-01T01:00:00.000000000', '2004-01-01T02:00:00.000000000',
        '2004-01-01T03:00:00.000000000', ...,
        '2021-08-29T22:00:00.000000000', '2021-08-29T23:00:00.000000000',
        '2021-08-30T00:00:00.000000000'],
       shape=(154824,), dtype='datetime64[ns]'),
 'target': array([ 7225.09,  6994.25,  6717.42, ..., 17114.34, 16091.05, 15081.16],
       shape=(154824,), dtype=float32)}

In [7]:
# future data that is known at prediction time (item ID, future timestamps, static and known covariates)
future_data

Dataset({
    features: ['id', 'timestamp'],
    num_rows: 8
})

In [8]:
future_data[0]

{'id': np.str_('COAST'),
 'timestamp': array(['2021-08-30T01:00:00.000000000', '2021-08-30T02:00:00.000000000',
        '2021-08-30T03:00:00.000000000', '2021-08-30T04:00:00.000000000',
        '2021-08-30T05:00:00.000000000', '2021-08-30T06:00:00.000000000',
        '2021-08-30T07:00:00.000000000', '2021-08-30T08:00:00.000000000',
        '2021-08-30T09:00:00.000000000', '2021-08-30T10:00:00.000000000',
        '2021-08-30T11:00:00.000000000', '2021-08-30T12:00:00.000000000',
        '2021-08-30T13:00:00.000000000', '2021-08-30T14:00:00.000000000',
        '2021-08-30T15:00:00.000000000', '2021-08-30T16:00:00.000000000',
        '2021-08-30T17:00:00.000000000', '2021-08-30T18:00:00.000000000',
        '2021-08-30T19:00:00.000000000', '2021-08-30T20:00:00.000000000',
        '2021-08-30T21:00:00.000000000', '2021-08-30T22:00:00.000000000',
        '2021-08-30T23:00:00.000000000', '2021-08-31T00:00:00.000000000'],
       dtype='datetime64[ns]')}

In [9]:
import numpy as np


def naive_forecast(y: list, horizon: int) -> dict[str, list]:
    # Make predictions for a single time series
    return {"predictions": [y[np.isfinite(y)][-1] for _ in range(horizon)]}

predictions_per_window = []
for window in task.iter_windows():
    past_data, future_data = window.get_input_data()
    predictions = [
        naive_forecast(ts[task.target], task.horizon) for ts in past_data
    ]
    predictions_per_window.append(predictions)

In [10]:
eval_summary = task.evaluation_summary(predictions_per_window, model_name="naive")
eval_summary

Casting the dataset: 100%|██████████| 8/8 [00:00<00:00, 5546.19 examples/s]


{'model_name': 'naive',
 'dataset_path': 'autogluon/chronos_datasets',
 'dataset_config': 'ercot',
 'horizon': 24,
 'num_windows': 2,
 'initial_cutoff': -48,
 'window_step_size': 24,
 'min_context_length': 1,
 'max_context_length': None,
 'seasonality': 1,
 'eval_metric': 'MASE',
 'extra_metrics': [],
 'quantile_levels': [],
 'id_column': 'id',
 'timestamp_column': 'timestamp',
 'target': 'target',
 'generate_univariate_targets_from': None,
 'known_dynamic_columns': [],
 'past_dynamic_columns': [],
 'static_columns': [],
 'task_name': 'ercot',
 'test_error': 7.301416219390507,
 'training_time_s': None,
 'inference_time_s': None,
 'dataset_fingerprint': '95b91121d95f89c8',
 'trained_on_this_dataset': False,
 'fev_version': '0.6.0rc3',
 'MASE': 7.301416219390507}

Evaluation summaries produced by different models on different tasks can be aggregated into a single table.

In [11]:
import pandas as pd

summaries = pd.read_csv("https://raw.githubusercontent.com/autogluon/fev/refs/heads/main/benchmarks/example/results/results.csv")
summaries.head()

,model_name,dataset_name,dataset_path,dataset_config,horizon,cutoff,lead_time,min_ts_length,max_context_length,seasonality,...,multiple_target_columns,past_dynamic_columns,excluded_columns,test_error,training_time_s,inference_time_s,dataset_fingerprint,trained_on_this_dataset,fev_version,MASE
0,auto_theta,chronos_datasets_monash_m1_yearly,autogluon/chronos_datasets,monash_m1_yearly,8,-8,1,9,NaN,1,...,NaN,[],[],4.241262,NaN,7.116432,296cc3ca5975e847,False,0.2.1,4.241262
1,auto_theta,chronos_datasets_monash_electricity_weekly,autogluon/chronos_datasets,monash_electricity_weekly,8,2013-01-01T00:00:00,1,9,NaN,1,...,NaN,[],[],1.428428,NaN,2.812927,1bf59473dbf463a3,False,0.2.1,1.428428
2,auto_theta,chronos_datasets_monash_electricity_weekly,autogluon/chronos_datasets,monash_electricity_weekly,8,2014-01-01T00:00:00,1,9,NaN,1,...,NaN,[],[],1.610647,NaN,6.573564,1bf59473dbf463a3,False,0.2.1,1.610647
3,auto_arima,chronos_datasets_monash_m1_yearly,autogluon/chronos_datasets,monash_m1_yearly,8,-8,1,9,NaN,1,...,NaN,[],[],3.993800,NaN,8.246975,296cc3ca5975e847,False,0.2.1,3.993800
4,auto_arima,chronos_datasets_monash_electricity_weekly,autogluon/chronos_datasets,monash_electricity_weekly,8,2013-01-01T00:00:00,1,9,NaN,1,...,NaN,[],[],1.720373,NaN,23.514658,1bf59473dbf463a3,False,0.2.1,1.720373


In [12]:
# Evaluation summaries can be provided as dataframes, dicts, JSON or CSV files
fev.leaderboard(summaries)

/tmp/ipykernel_3068333/606848490.py:2: UserWarning: Columns ['known_dynamic_columns', 'static_columns'] are missing from summaries, filling them with None
  fev.leaderboard(summaries)


,gmean_relative_error,avg_rank,avg_inference_time_s,median_inference_time_s,avg_training_time_s,median_training_time_s,training_corpus_overlap,num_failures
model_name,,,,,,,,
auto_theta,0.874455,2.000000,5.500974,6.573564,NaN,NaN,0.0,0
auto_arima,0.887336,2.000000,21.798952,23.514658,NaN,NaN,0.0,0
auto_ets,0.951193,2.666667,0.736867,0.741776,NaN,NaN,0.0,0
seasonal_naive,1.000000,3.333333,0.003800,0.004139,NaN,NaN,0.0,0


The `leaderboard` method not only summarizes the results into a single table, but also ensures that all task definitions & dataset fingerprints match across different models. This ensures that the scores are comparable and the comparison is fair.